In [18]:
import pandas as pd
import numpy as np
import re
import os, sys
import tiktoken
sys.path.append("..")

In [76]:
pd.set_option('display.max_colwidth', None)  # None means no limit on column width
pd.set_option('display.width', 300)  

sci = pd.read_parquet("../datasets/science_parallel.parquet")
sci.head()

title_en                                                                                 title_sr  \
0  The origins and development of the architectural form of Roman Catholic churches in Vojvodina in the 18th and 19th century  Poreklo i razvoj arhitektonske forme rimokatoličkih crkava XVIII i XIX veka u Vojvodini   
1                                                         Belgrade's General Plan 1923: a comparison of planned to completed.                         Generalni plan Beograda 1923 komparacija planiranog i ostvarenog   
2                           The importance and role of pedestrian space network i generating the competitive identity of city      Značaj i uloga mreže pešačkih prostora u generisanju kompetitivnog identiteta grada   
3                                                                                 Adaptive principles in architectural design                                        Adaptivni principi u arhitektonskom projektovanju   
4                       Transformation of vernacular architecture on the Vrmac peninsula - Bay of Kotor during the XX century        Transformacija narodne arhitekture na poluostrvu Vrmac - Boka Kotorska u 20. veku   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [77]:
sci['abstract_en'] = sci['abstract_en'].replace('', np.nan)
sci['abstract_sr'] = sci['abstract_sr'].replace('', np.nan)
sci_cleaned = sci.dropna(subset=['abstract_en', 'abstract_sr'], ignore_index=True)
sci_cleaned.shape

(9962, 6)

In [13]:
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """
    Calculates the number of tokens in a string based on a specified encoding.

    The function uses the specified encoding to tokenize the input string and returns the number of tokens generated.

    Parameters:
    string (str): The input string to be tokenized.
    encoding_name (str): The name of the encoding to use for tokenization.

    Returns:
    int: The number of tokens in the input string based on the specified encoding.

    Example:
    >>> string = "This is a sample sentence."
    >>> encoding_name = "cl100k_base"
    >>> num_tokens = num_tokens_from_string(string, encoding_name)
    >>> print(num_tokens)
    """
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [16]:
tokens_sr_processed = [num_tokens_from_string(i, "cl100k_base") for i in sci['abstract_sr']]

In [19]:
print(f"Mean (tokens): {np.mean(tokens_sr_processed)}")
print(f"Max length (tokens): {np.max(tokens_sr_processed)}")
print(f"Min length (tokens): {np.min(tokens_sr_processed)}")


Mean (tokens): 746.1970050417639
Max length (tokens): 11258
Min length (tokens): 0


In [22]:
for i,idx in enumerate(tokens_sr_processed):
    if i == 0:
        print(idx)

1644


In [24]:
sci["abstract_sr"][1644]

'Uvod: Inflamatorna bolest creva (IBC), čija su dva glavna klinička oblika ulcerozni kolitis (UK) i Kronova bolest (KB), predstavlja heterogenu grupu bolesti hroničnog toka koje se karakterišu ponavljanim epizodama inflamacije u gastrointestinalnom traktu. Iako etiopatogeneza IBC još uvek nije razjašnjena, smatra se da nastaje kao posledica aberantnog imunskog odgovora na mikroorganizme u crevima koji kod genetski predisponirane osobe dovodi do hronične inflamacije i oštećenja crevne mukoze. Sa napretkom molekularne genetike identifikovan je veliki broj polimorfizama pojedinačnih nukleotida u različitim genima koji bi mogli da se koriste kao biomarkeri u IBC i pomognu u dijagnostici, klasifikaciji bolesti, prognozi i praćenju terapijskog odgovora. Među njima se posebno ističu polimorfizmi u genima koji kodiraju proinflamatorne citokine (TNF), antiinflamatorne citokine (IL-10) i citokine važne za razvoj i funkciju Th1 i Th17 ćelija (IL-12 i IL-23 i njihove receptore), kao i polimorfizmi

In [72]:
def tokenize_and_truncate(text, encoding_name="cl100k_base", max_tokens=512):
    encoding = tiktoken.get_encoding(encoding_name)
    tokens = encoding.encode(text)
    num_tokens = len(tokens)
    if num_tokens > max_tokens:
        tokens = tokens[:max_tokens]  # Truncate to 512 tokens
    processed_abstract = encoding.decode(tokens)  # Decode back to text
    if not processed_abstract.endswith('.'):
        last_period_index = processed_abstract.rfind('.')
        if last_period_index != -1:
            processed_abstract = processed_abstract[:last_period_index + 1]  # Keep everything up to the last "."
            
    return processed_abstract


In [90]:
sci_cleaned['processed_abstract_sr'] = sci_cleaned["abstract_sr"].apply(tokenize_and_truncate)
sci_cleaned['processed_abstract_en'] = sci_cleaned["abstract_en"].apply(tokenize_and_truncate)

/tmp/ipykernel_21975/2407012175.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sci_cleaned['processed_abstract_sr'] = sci_cleaned["abstract_sr"].apply(tokenize_and_truncate)
/tmp/ipykernel_21975/2407012175.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sci_cleaned['processed_abstract_en'] = sci_cleaned["abstract_en"].apply(tokenize_and_truncate)


In [91]:
sci_cleaned.head()

title_en                                                                                 title_sr  \
0  The origins and development of the architectural form of Roman Catholic churches in Vojvodina in the 18th and 19th century  Poreklo i razvoj arhitektonske forme rimokatoličkih crkava XVIII i XIX veka u Vojvodini   
1                                                         Belgrade's General Plan 1923: a comparison of planned to completed.                         Generalni plan Beograda 1923 komparacija planiranog i ostvarenog   
2                           The importance and role of pedestrian space network i generating the competitive identity of city      Značaj i uloga mreže pešačkih prostora u generisanju kompetitivnog identiteta grada   
3                                                                                 Adaptive principles in architectural design                                        Adaptivni principi u arhitektonskom projektovanju   
4                       Transformation of vernacular architecture on the Vrmac peninsula - Bay of Kotor during the XX century        Transformacija narodne arhitekture na poluostrvu Vrmac - Boka Kotorska u 20. veku   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [85]:
tokens_sr_processed = [num_tokens_from_string(i, "cl100k_base") for i in sci_cleaned['processed_abstract_sr']]

idxs_sr = []
for idx, i in enumerate(tokens_sr_processed):
    if i == 0:
        idxs_sr.append(idx)

print(f"Mean (tokens): {np.mean(tokens_sr_processed)}")
print(f"Max length (tokens): {np.max(tokens_sr_processed)}")
print(f"Min length (tokens): {np.min(tokens_sr_processed)}")
print(f"Missing: {len(idxs_sr)}")

Mean (tokens): 431.1939369604497
Max length (tokens): 512
Min length (tokens): 16
Missing: 0


In [89]:
tokens_en = [num_tokens_from_string(i, "cl100k_base") for i in sci_cleaned['processed_abstract_en']]

idxs_en = []
for idx_en, i in enumerate(tokens_en):
    if i == 0:
        idxs_en.append(idx)

print(f"Mean (tokens): {np.mean(tokens_en)}")
print(f"Max length (tokens): {np.max(tokens_en)}")
print(f"Min length (tokens): {np.min(tokens_en)}")
print(f"Missing: {len(idxs_en)}")

Mean (tokens): 370.8576591045975
Max length (tokens): 512
Min length (tokens): 8
Missing: 0


In [93]:
def extract_first_sentence(abstract):
    return abstract.split('.')[0] + '.' if '.' in abstract else abstract

In [94]:
df_sentences = pd.DataFrame({
    'sentence_en': sci_cleaned['processed_abstract_en'].apply(extract_first_sentence),
    'sentence_sr': sci_cleaned['processed_abstract_sr'].apply(extract_first_sentence)
})

In [96]:
df_sentences.sample(30)

,sentence_en,sentence_sr
4159,"The doctoral dissertation Time, space and body in Zivojin Pavlovic’s fiction and film looks at the way in which these three elements – time, space and body – are connected in constructing the narrative in both literature and film.","U doktorskoj disertaciji Vreme, prostor i telo u prozi i filmu Živojina Pavlovića pokazano je na koji način se ova tri elementa povezuju tokom izgradnje narativa u obe umetnosti – književnoj i filmskoj."
361,The aim of presented PhD thesis was research of biodeterioration of cultural heritage objects in Serbia.,Cilj istraživanja doktorske disertacije je bio proučavanje biodeterioracije objekata kulturne baštine u Srbiji.
2097,Numerous abandoned mining operations and limited funding for the recultivation and remediation of degraded soils have motivated the development of a method for prioritizing locations on which further investigations should focus.,Veliki broj napuštenih rudarskih radova i ograničena sredstva za rekultivaciju i remedijaciju degradiranih prostora podstakle su razvoj metoda za izdvajanje prioritetnih lokacija na koje je potrebno usmeriti dalja istraživanja.
8623,The dissertation aimed to develop a population model of intra-thyroidal 131I biokinetics as well as a dynamic model of the 131I therapy outcome probability in patients with benign thyroid diseases using the nonlinear mixed-effects modelling approach.,"Cilj disertacije je bio da se primenom pristupa nelinearnog modelovanja kombinovanih efekata razvije populacioni model intratireoidne biokinetike 131I, kao i dinamički model verovatnoće ishoda terapije 131I kod pacijenata sa benignim oboljenjima štitaste žlezde."
4267,"Curly dock (Rumex crispus, Polygonaceae) is a wild perennial herbaceous plant, which products are described as a rich source of phenolic compounds.","Štavelj (Rumex crispus, Polygonaceae) je višegodišnja zeljasta biljka, koja predstavlja bogat izvor fenolnih komponenti."
5079,"The goal of the doctoral dissertation entitled „History and Politics in the Novels of Nadine Gordimer“ is to understand the impact of history and politics on the literary oeuvre of Nadine Gordimer (1923), a South African writer, the winner of the Nobel Prize for literature, as well as many other literary awards and honorable academic titles.","Predmet istraživanja disertacije pod nazivom „Istorija i politika u romanima Nejdin Gordimer“, jeste ispitivanje uticaja istorije i politike na književno delo Nejdin Gordimer (1923), južnoafričke književnice, dobitnice Nobelove nagrade za književnost, kao i mnogih drugih književnih nagrada i akademskih priznanja."
7530,The research in this paper was performed in the forests of sessile oak (Quercetum montanum Cer.,Istraživanja u ovom radu su izvršena u šumama hrasta kitnjaka (Quercetum montanum Cer.
2927,"The subject of this PhD thesis is examination of electrochemical activity of amlodipine and nifedipine, a 1,4-dihydropyridine type of Ca2+ channel blockers used in treatment of cardiovascular diseases.","Predmet ove doktorske disertacije je ispitivanje elektrohemijske aktivnosti amlodipina i nifedipina odnosno proučavanje odabranih 1,4-dihidropiridinskih lekova koji se kao blokatori Ca2+-kanala koriste u terapiji kardiovaskularnih oboljenja."
9484,"Modern European societies are characterized by a democratic political regime and capitalism as an economic system of production and distribution in which workers’ participation finds its place as a political, legal and social reality.","Moderna evropska društva karakteriše demokratski politički režim i kapitalizam kao ekonomski sistem proizvodnje i raspodele u kome participacija radnika nalazi svoje mesto kao politička, pravna i socijalna realnost."
9653,"Тhe indicators of autonomic control alterations, including excessive fat accumulation, are affected by several physiological and pathophysiological attributes.",Indikatori nepovoljne autonomne kontrole su pod uticajem nekoliko fizioloških i patofizioloških faktora.


In [98]:
nesto = pd.read_parquet("../datasets/sr_en_parallel_10k.parquet")
nesto

,input_engl,input_srb
28839,How can you use technology to improve your customer service?,Kako možete koristiti tehnologiju da poboljšate svoju korisničku uslugu?
28301,"Sort the numbers 15, 11, 9, 22.","Poređaj brojeve 15, 11, 9, 22."
25859,Create a list of 8 questions to ask prospective online tutors.,Napravite listu od 8 pitanja koja ćete postaviti budućim onlajn učiteljima.
16869,Rewrite the given poem so that it rhymes,Prepiši datu pesmu tako da se rimuje
48319,Provide a realistic context for the following sentence.,Navedite realističan kontekst za sledeću rečenicu.
...,...,...
13713,"Find the equation of the line which passes through (2, 4) and is perpendicular to the line -x + y = 3","Naći jednačinu prave koja prolazi kroz (2, 4) i koja je okomita na pravu -k + i = 3"
27349,What does the phrase 'give-and-take' mean?,Šta znači fraza 'daj-uzmi'?
29025,Change the language in this sentence from first person to third person.,Promenite jezik u ovoj rečenici iz prvog u treće lice.
46455,"Given a text input, summarize the key points into a shorter output.","S obzirom na unos teksta, sumirajte ključne tačke u kraći rezultat."
